# This tutorial shows how to get more precise result with keywords while calling a result from a solution object

### Get the result object

In [1]:
from ansys.dpf import post
solution = post.load_solution("d:/rst/static_twobodies_ns.rst")

server started at --address 10.110.2.65 --port 50052


### Get the keyword list

In [2]:
post.print_available_keywords()

element_scoping: list, int or dpf.core.Scoping
grouping: str. Use post.grouping.(...) as helper.
location: str. Use post.locations.(...) as helper.
mapdl_grouping: int. Write 186 to get mapdl_elements solid_186.
named_selection: str. Name of named_selection.
node_scoping: list, int or dpf.core.Scoping
set: int
time: float
time_scoping: list, int or dpf.core.Scoping



### Use a scoping on result

In [3]:
displacement_result = solution.displacement(location = post.locations.nodal, node_scoping = [1, 2, 3]) #default location is nodal
displacement = displacement_result.vector

In [4]:
displacement.get_data_at_field(0)

array([[ 0.00379735,  0.04473428,  0.00831521],
       [ 0.00254443,  0.04473239,  0.00067533],
       [ 0.00129122,  0.04473354, -0.00696479]])

In [5]:
stress_with_elem_scop_result = solution.stress(location = post.locations.elemental_nodal, element_scoping = [1])
stress_with_elem_scop = stress_with_elem_scop_result.tensor

In [6]:
stress_with_elem_scop.get_data_at_field(0)

array([[-7.76947840e+08,  2.30038374e+09,  1.79927373e+09,
        -5.30445504e+08, -6.89238528e+08,  1.00718451e+09],
       [-8.34727552e+08,  5.74724864e+08,  4.41359923e+09,
         6.39719080e+07,  2.51828634e+09,  1.39846362e+09],
       [-5.50896640e+08,  7.95577856e+08,  5.93578957e+09,
         2.70925888e+08, -2.62505232e+08,  2.07582515e+09],
       [-6.12245504e+08,  1.05938989e+09,  4.76268544e+09,
         1.49492390e+09,  7.03467072e+08,  1.08908582e+09],
       [-2.09427174e+09, -6.03949312e+09,  9.34414438e+09,
         3.42422426e+09,  2.58427725e+09,  5.90829261e+09],
       [ 1.95679718e+09, -1.90205555e+09,  1.60553646e+10,
        -8.90331584e+08, -2.48762189e+09,  5.10888960e+09],
       [-3.04948582e+09,  1.24922099e+09,  1.08264899e+10,
         6.53856384e+08,  1.12002701e+09,  4.68143821e+09],
       [ 6.68166272e+08, -6.60018176e+08,  1.53434348e+10,
        -1.36846669e+09, -2.75138440e+07,  3.22212838e+09]])

### Use a named selection on result (MAPDL named_selection name must be written using UPPER CASE)

In [7]:
stress_on_ns_result = solution.stress(location = post.locations.elemental_nodal, named_selection="SELECTION") #MAPDL named_selection name must be written is upper case
stress_on_ns = stress_on_ns_result.tensor

In [8]:
stress_on_ns.num_fields

1

In [9]:
len(stress_on_ns[0])

24480

### Get a subresult

In [10]:
disp_x = displacement_result.x

In [11]:
stress_yz = stress_with_elem_scop_result.yz

In [12]:
stress_principal_1 = stress_on_ns_result.principal_3

### Filter on a time / time_scoping / a set

In [13]:
print(solution.time_freq_support)

Time/Frequency Info:
	Number of sets: 1

With complex values
 
 Cumulative      Time (s)       Loadstep     Substep   
     1             1.0             1            1      



In [14]:
stress_on_time_1s_result = solution.stress(time = 1.)
stress_on_time_1s = stress_on_time_1s_result.tensor

In [15]:
displacement_on_set_1_result = solution.displacement(set = 1)
displacement_on_set_1 = displacement_on_set_1_result.vector

In [16]:
elastic_strain_with_time_scoping_result = solution.elastic_strain(time_scoping = [1, 3])
elastic_strain_with_time_scoping = elastic_strain_with_time_scoping_result.tensor

### Make a grouping

In [17]:
displacement_result = solution.displacement(grouping = post.grouping.by_el_shape)
displacement_by_el_shape = displacement_result.vector

In [18]:
stress_result = solution.stress(mapdl_grouping = 186) #will filter only MAPDL elements of type solid 186
stress_on_solid_186 = stress_result.tensor

## Manipulate the result object and change its definition

In [19]:
print(stress_on_ns_result)

Stress. 
Tensor object. 

Object properties are: 
 - location = ElementalNodal
 - named_selection = SELECTION



In [20]:
stress_on_ns_result.definition.location

'ElementalNodal'

In [25]:
stress_on_ns_result.definition.location = post.locations.elemental
stress_on_ns_result.definition.time = 1.
stress_on_ns_elemental = stress_on_ns_result.tensor

In [26]:
print(stress_on_ns_result)

Stress. 
Tensor object. 

Object properties are: 
 - location = Elemental
 - named_selection = SELECTION
 - time = 1.0



## Use the misc. results

##### Same keywords can be used here. For complex result, the keyword "phase" (float) can also be used.

In [32]:
stress_ratio = solution.misc.elemental_stress_ratio(node_scoping = [1, 32], time = 1.)
print(stress_ratio)

Stress result.

The result is computed thanks to dpf.core.Operator objects, that are chained together regarding the following list: 
- ENL_SRAT: Result operator. Compute the wanted result
- to_elemental_fc: This operator will compute the elemental averaging of a fields container.



In [28]:
stress_ratio.num_fields

1

In [ ]:
stress_ratio.max_data